### Loading IFC data  
To explore the IFC model, we use the [IfcOpenShell](https://docs.ifcopenshell.org/ifcopenshell-python/installation.html) module, you will have to install it again when using a virtual environment. <br>


In [ ]:
import ifcopenshell as ios

model = ios.open("data/HeartbreakHotel.ifc")

### Iterating over the IfcStair instances 
We want to iterate over all stair instances and their properties and drop them into a list.<br> 
The same applies to relational object instances, as these define other objects that may be related to a stair flight, such as railings... 

In [ ]:
ifcstairs = model.by_type('IfcStair')
relations = model.by_type('IfcRelAggregates')
print(str(len(ifcstairs))+' IfcStairs found')

### Drilling down the IfcModel in search for the IfcStair properties
For convenience, we use a custom Python class that represents a stair concept. 

In [ ]:
from stair import Stair
stair = []
for ifcstair in ifcstairs:
    if hasattr(ifcstair, "IsDefinedBy"):
        for rel in ifcstair.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties"):
                property_set = rel.RelatingPropertyDefinition
                if property_set.is_a("IfcPropertySet") and property_set.Name == "Pset_StairCommon":
                    if hasattr(property_set, "HasProperties"):
                        for property in property_set.HasProperties:
                            if property.is_a("IfcPropertySingleValue"):
                                if property.Name == "RiserHeight":
                                    riserheight = round(property.NominalValue.wrappedValue)
                                if property.Name == "TreadLength":
                                    threadlength = round(property.NominalValue.wrappedValue)
    numberofrailings=0
    for rel in relations:
        for related_element in rel.RelatedObjects:
            if related_element.is_a("IfcRailing"):
                if rel.RelatingObject.GlobalId == ifcstair.GlobalId:
                    numberofrailings +=1
    
    new_stair = Stair(ifcstair.Name, riserheight, threadlength, numberofrailings)
    stair.append(new_stair)
    
print(len(stair), "stairs created.")

### Dumping the stair data
Finally, we write the list of stair objects to a json file. In an intermediate step, we transform the list into a dictionary. Note that the json module has to be imported first. 

In [ ]:

import json

stairs_dict = [stair.__dict__ for stair in stair]
with open("data/stairs.json", "w") as json_file:
    json.dump(stairs_dict, json_file, indent=4)